## run this cell before starting spark cluster connection (corrects module error)

In [64]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [65]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [66]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [67]:
df_lookup = spark.read.csv("/user/sprenner/lookup.csv", sep="\t")

In [68]:
#df_lookup.describe().toPandas()

In [69]:
#df_lookup.limit(10).toPandas()

In [70]:
df_lookup_renamed = df_lookup.toDF('SCOPE', 'NAME', 'CHILD_SCOPE', 'CHILD_NAME', 'DID_TYPE', 'CHILD_TYPE', 
                                   'LENGTH', 'BYTES', 'ADLER32', 'MD5', 'RULE_EVALUATION', 'UPDATED_AT', 
                                   'CREATED_AT', 'EVENTS')

In [71]:
#df_lookup_renamed.limit(10).toPandas()

In [72]:
#df_lookup_renamed.drop('ADLER32','MD5','RULE_EVALUATION')

In [73]:
#df_lookup.write.csv('/user/sprenner/lookup.csv', sep='\t')

In [74]:
df_april = spark.read.json("/user/sprenner/training_official_april.json")

In [75]:
df_april = df_april.withColumn("join_key_april", concat_ws(':', df_april["scope"], df_april["filename"]))
df_april = df_april.withColumn("fileScope", df_april["scope"])
df_april = df_april.drop("scope")
df_april.createOrReplaceTempView("april")
df_lookup_renamed = df_lookup_renamed.withColumn("join_key_lookup", concat_ws(':', df_lookup_renamed["CHILD_SCOPE"], df_lookup_renamed["CHILD_NAME"]))
df_joined = df_april.join(df_lookup_renamed, df_april.join_key_april == df_lookup_renamed.join_key_lookup, "left_outer")

df_joined.createOrReplaceTempView("joined")
#df_joined_check = spark.sql("SELECT join_key_lookup, COUNT(*)\
#                             FROM joined\
#                             GROUP BY join_key_lookup\
#                             ORDER BY 2 DESC")
#df_joined_check.limit(20).toPandas()

In [76]:
#df_joined_check_2 = spark.sql("SELECT join_key_april, join_key_lookup, SCOPE, DID_TYPE\
#                                 FROM joined\
#                                 WHERE join_key_lookup == 'None' OR join_key_lookup IS NULL")
#df_joined_check_2.limit(10).toPandas()

In [77]:
#df_joined_check_3 = spark.sql("SELECT *\
#                                 FROM joined\
#                                 WHERE join_key_lookup == 'mc16_13TeV:HITS.10701323._005364.pool.root.1'")
#df_joined_check_3.limit(10).toPandas()

In [78]:
#df_joined_check_4 = spark.sql("SELECT NAME, COUNT(*)\
#                                 FROM joined\
#                                 WHERE join_key_lookup == 'mc16_13TeV:HITS.10701323._005364.pool.root.1' AND DID_TYPE == 'D'\
#                                 GROUP BY NAME\
#                                 ORDER BY 2 DESC")
#df_joined_check_4.limit(10).toPandas()

In [79]:
#df_joined_check_4 = spark.sql("SELECT SCOPE, COUNT(*)\
#                                 FROM joined\
#                                 WHERE join_key_lookup == 'mc16_13TeV:HITS.10701323._005364.pool.root.1' AND DID_TYPE == 'D'\
#                                 GROUP BY SCOPE\
#                                 ORDER BY 2 DESC")
#df_joined_check_4.limit(10).toPandas()

In [80]:
#df_joined.printSchema()

In [81]:
#df_joined_check_5 = spark.sql("SELECT SCOPE, COUNT(*)\
#                                 FROM joined\
#                                 WHERE join_key_lookup == 'mc16_13TeV:HITS.10701323._000012.pool.root.1' AND DID_TYPE == 'D'\
#                                 GROUP BY SCOPE\
#                                 ORDER BY 2 DESC")
#df_joined_check_5.limit(10).toPandas()

In [82]:
#df_joined.agg(*(countDistinct(col(c)).alias(c) for c in df_joined.columns)).toPandas()

In [83]:
df_joined_check_6 = spark.sql("SELECT *\
                                 FROM joined\
                                 WHERE SCOPE != 'panda' AND DID_TYPE == 'D'")
#df_joined_check_6.agg(*(countDistinct(col(c)).alias(c) for c in df_joined_check_6.columns)).toPandas()

In [84]:
#df_joined.describe().toPandas()

In [85]:
#df_joined_check_6.describe().toPandas()

In [86]:
df_joined_check_6.createOrReplaceTempView("joined6")
#df_joined_check_7 = spark.sql("SELECT join_key_lookup, COUNT(*)\
#                             FROM joined6\
#                             GROUP BY join_key_lookup\
#                             ORDER BY 2 DESC")
#df_joined_check_7.limit(20).toPandas()

In [87]:
#df_joined.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_joined.columns]).limit(10).toPandas()

In [88]:
#df_joined_check_6.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_joined_check_6.columns]).limit(10).toPandas()

In [89]:
df_joined_check_6 = df_joined_check_6.drop('LENGTH','ADLER32','MD5','RULE_EVALUATION','EVENTS')

In [90]:
#df_joined_check_6.write.csv('/user/sprenner/lookup_clean_datasets.csv', sep='\t')

In [91]:
df_joined_check_6.createOrReplaceTempView("joined6")


df_lookup_renamed.createOrReplaceTempView("lookup_renamed")
multi_datasets = spark.sql("SELECT *\
                      FROM (SELECT join_key_april AS join_key_april_multi, COUNT(DISTINCT NAME) AS distinct_datasets\
                      FROM joined6 \
                      GROUP BY join_key_april \
                      ORDER BY distinct_datasets DESC)\
                  WHERE distinct_datasets > 1")
multi_datasets.toPandas()

join_key_april_multi  distinct_datasets
0     mc16_13TeV:HITS.10701323._008539.pool.root.1         11               
1     mc16_13TeV:AOD.11264194._000001.pool.root.1          11               
2     mc16_13TeV:AOD.11264194._000002.pool.root.1          10               
3     mc16_13TeV:AOD.11264194._000005.pool.root.1          10               
4     mc16_13TeV:AOD.11264194._000059.pool.root.1          10               
5     mc16_13TeV:AOD.11264194._000003.pool.root.1          10               
6     mc16_13TeV:AOD.11264194._000082.pool.root.1          10               
7     mc16_13TeV:AOD.11264194._000026.pool.root.1          10               
8     mc16_13TeV:AOD.11264194._000065.pool.root.1          10               
9     mc15_13TeV:EVNT.04972714._000021.pool.root.1         10               
10    mc16_13TeV:AOD.11264194._000009.pool.root.1          10               
11    mc16_13TeV:AOD.11264194._000018.pool.root.1          10               
12    mc16_13TeV:AOD.11264194._000046.pool.root.1          10               
13    mc16_13TeV:AOD.11264194._000022.pool.root.1          10               
14    mc16_13TeV:AOD.11264194._000030.pool.root.1          10               
15    mc16_13TeV:AOD.11264194._000017.pool.root.1          10               
16    mc16_13TeV:AOD.11264194._000056.pool.root.1          10               
17    mc16_13TeV:AOD.11264194._000083.pool.root.1          10               
18    mc16_13TeV:AOD.11264194._000036.pool.root.1          10               
19    mc16_13TeV:AOD.11264194._000037.pool.root.1          10               
20    mc16_13TeV:AOD.11264194._000015.pool.root.1          10               
21    mc16_13TeV:AOD.11264194._000058.pool.root.1          10               
22    mc16_13TeV:AOD.11264194._000043.pool.root.1          10               
23    mc16_13TeV:AOD.11264194._000075.pool.root.1          10               
24    mc16_13TeV:AOD.11264194._000089.pool.root.1          10               
25    mc16_13TeV:AOD.11264194._000061.pool.root.1          10               
26    mc16_13TeV:AOD.11264194._000004.pool.root.1          10               
27    mc16_13TeV:AOD.11264194._000085.pool.root.1          10               
28    mc16_13TeV:AOD.11264194._000062.pool.root.1          10               
29    mc16_13TeV:AOD.11264194._000032.pool.root.1          10               
...                                           ...          ..               
3110  mc16_13TeV:DAOD_SUSY5.13463372._000049.pool.root.1   2                
3111  mc16_5p44TeV:AOD.13641049._000233.pool.root.1        2                
3112  mc15_13TeV:EVNT.13705598._000046.pool.root.1         2                
3113  mc15_valid:EVNT.13637531._000056.pool.root.1         2                
3114  mc15_valid:EVNT.13664312._000518.pool.root.1         2                
3115  mc15_valid:EVNT.13664314._000160.pool.root.1         2                
3116  mc15_valid:EVNT.13664314._000466.pool.root.1         2                
3117  mc15_valid:EVNT.13664314._000608.pool.root.1         2                
3118  mc15_valid:EVNT.13664314._001011.pool.root.1         2                
3119  mc15_valid:EVNT.13664314._001064.pool.root.1         2                
3120  mc15_valid:EVNT.13785174._001115.pool.root.1         2                
3121  mc16_13TeV:AOD.13642632._001476.pool.root.1          2                
3122  mc16_13TeV:DAOD_SUSY5.13462844._000025.pool.root.1   2                
3123  mc16_13TeV:DAOD_SUSY5.13462844._000026.pool.root.1   2                
3124  mc16_13TeV:DAOD_SUSY5.13462844._000124.pool.root.1   2                
3125  mc16_13TeV:NTUP_PILEUP.13658010._000010.pool.root.1  2                
3126  mc16_5p44TeV:AOD.13641049._000061.pool.root.1        2                
3127  mc16_5p44TeV:AOD.13641049._000197.pool.root.1        2                
3128  mc15_14TeV:AOD.13709303._001479.pool.root.1          2                
3129  mc15_14TeV:AOD.13709303._001890.pool.root.1          2                
3130  mc15_valid:EVNT.13664314._0

In [92]:
single_datasets = spark.sql("SELECT *\
                      FROM (SELECT join_key_april AS join_key_april_single, COUNT(DISTINCT NAME) AS distinct_datasets\
                      FROM joined6 \
                      GROUP BY join_key_april \
                      ORDER BY distinct_datasets DESC)\
                  WHERE distinct_datasets < 2")
single_datasets.toPandas()

join_key_april_single  \
0       data15_13TeV:DAOD_EXOT8.13466291._000030.pool.root.1     
1       data15_13TeV:DAOD_EXOT8.13466382._000037.pool.root.1     
2       data15_13TeV:DAOD_EXOT8.13466420._000001.pool.root.1     
3       data15_13TeV:DAOD_HIGG3D1.12645203._000016.pool.root.1   
4       data15_13TeV:DAOD_HIGG3D1.12645292._000018.pool.root.1   
5       data15_13TeV:DAOD_HIGG3D1.12645304._000011.pool.root.1   
6       data15_13TeV:DAOD_HIGG3D1.12645319._000044.pool.root.1   
7       data15_13TeV:DAOD_HIGG3D1.12645368._000038.pool.root.1   
8       data15_13TeV:DAOD_HIGG3D1.12645495._000060.pool.root.1   
9       data15_13TeV:DAOD_HIGG3D1.12645495._000062.pool.root.1   
10      data15_13TeV:DAOD_HIGG3D1.12645495._000150.pool.root.1   
11      data15_13TeV:DAOD_HIGG3D1.12645561._000022.pool.root.1   
12      data15_13TeV:DAOD_HIGG3D1.12645561._000044.pool.root.1   
13      data15_13TeV:DAOD_HIGG3D1.12645561._000055.pool.root.1   
14      data15_13TeV:DAOD_HIGG3D1.12645609._000028.pool.root.1   
15      data15_13TeV:DAOD_HIGG4D4.10292387._000007.pool.root.1   
16      data15_13TeV:DAOD_HIGG4D4.10292395._000042.pool.root.1   
17      data15_13TeV:DAOD_HIGG4D4.10292395._000074.pool.root.1   
18      data15_13TeV:DAOD_HIGG4D4.10292472._000004.pool.root.1   
19      data15_13TeV:DAOD_HIGG4D4.10292526._000028.pool.root.1   
20      data15_13TeV:DAOD_HIGG4D4.10292685._000017.pool.root.1   
21      data15_13TeV:DAOD_HIGG4D4.10292685._000057.pool.root.1   
22      data15_13TeV:DAOD_HIGG4D4.10292685._000117.pool.root.1   
23      data15_13TeV:DAOD_HIGG4D4.10292984._000084.pool.root.1   
24      data15_13TeV:DAOD_HIGG4D4.10293048._000122.pool.root.1   
25      data15_13TeV:DAOD_HIGG4D4.10293127._000166.pool.root.1   
26      data15_13TeV:DAOD_HIGG4D4.10293127._000297.pool.root.1   
27      data15_13TeV:DAOD_HIGG4D4.10293127._000320.pool.root.1   
28      data15_13TeV:DAOD_HIGG4D4.10293180._000177.pool.root.1   
29      data15_13TeV:DAOD_HIGG4D4.10293180._000371.pool.root.1   
...                                                        ...   
526926  mc16_13TeV:NTUP_PILEUP.13422840._000112.pool.root.1      
526927  mc16_13TeV:NTUP_PILEUP.13425173._000001.pool.root.1      
526928  mc16_13TeV:NTUP_PILEUP.13439815._000001.pool.root.1      
526929  mc16_13TeV:NTUP_PILEUP.13440638._000001.pool.root.1      
526930  mc16_13TeV:NTUP_PILEUP.13460216._000011.pool.root.1      
526931  mc16_13TeV:NTUP_PILEUP.13479916._000002.pool.root.1      
526932  mc16_13TeV:NTUP_PILEUP.13514667._000001.pool.root.1      
526933  mc16_13TeV:NTUP_PILEUP.13567451._000019.pool.root.1      
526934  mc16_13TeV:NTUP_PILEUP.13567493._000006.pool.root.1      
526935  mc16_13TeV:NTUP_PILEUP.13567523._000039.pool.root.1      
526936  mc16_13TeV:NTUP_PILEUP.13567556._000013.pool.root.1      
526937  mc16_13TeV:NTUP_PILEUP.13575778._000001.pool.root.1      
526938  mc16_13TeV:NTUP_PILEUP.13640623._000190.pool.root.1      
526939  mc16_13TeV:NTUP_PILEUP.13644267._000017.pool.root.1      
526940  mc16_13TeV:NTUP_PILEUP.13644267._000054.pool.root.1      
526941  mc16_13TeV:NTUP_PILEUP.13644281._000018.pool.root.1      
526942  mc16_13TeV:RDO.11426804._000001.pool.root.1              
526943  mc16_13TeV:RDO.13121605._000059.pool.root.1              
526944  mc16_13TeV:log.12627706._000036.job.log.tgz.1            
526945  mc16_5TeV:AOD.13055721._001907.pool.root.1               
526946  mc16_5p44TeV:AOD.13641042._000099.pool.root.1            
526947  mc16_5p44TeV:AOD.13641042._000390.pool.root.1            
526948  mc16_5p44TeV:AOD.13641044._000056.pool.root.1            
526949  mc16_5p44TeV:AOD.13641044._000110.pool.root.1            
526950  mc16_5p44TeV:log.13482647._000198.job.log.tgz.1          
526951  mc16_5p44TeV:log.13634586._000109.job.log.tgz.1          
526952  mc16_5p44TeV:log.13634586._000221.job.log.tgz.1          
526953  mc16_valid:log.11892265._000068.job.log.tgz.1            
526954  mc16_valid:log.11892265._000105.job.log.tgz.1            
526955  mc16_va

In [108]:
single_datasets.createOrReplaceTempView("single_datasets")
datasets = spark.sql("SELECT * \
                      FROM april LEFT JOIN single_datasets ON april.join_key_april == single_datasets.join_key_april_single")
datasets.limit(10).toPandas()

account  accountHash clientState  clientStateHash  \
0  lhavener  1677896849   DONE       -878527175         
1  jsabater  1852428352   DONE       -878527175         
2  jsabater  1852428352   DONE       -878527175         
3  jsabater  1852428352   DONE       -878527175         
4  jsabater  1852428352   DONE       -878527175         
5  jsabater  1852428352   DONE       -878527175         
6  tofitsch  71670540     DONE       -878527175         
7  lhavener  1677896849   DONE       -878527175         
8  adurglis  813774196    DONE       -878527175         
9  adurglis  813774196    DONE       -878527175         

                                                                                                          dataset  \
0  mc15_5TeV.420000.Hijing_PbPb_5p02TeV_MinBias_Flow_JJFV6.merge.AOD.e4962_s2891_s2921_r7899_r7676_tid08671884_00   
1  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709              
2  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768              
3  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709              
4  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768              
5  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768              
6  mc15_14TeV.147911.Pythia8_AU2CT10_jetjet_JZ1W.recon.AOD.e2403_s3142_s3143_r9589                                  
7  mc15_5TeV.420000.Hijing_PbPb_5p02TeV_MinBias_Flow_JJFV6.merge.AOD.e4962_s2891_s2921_r7899_r7676_tid08671884_00   
8  mc15_14TeV.415013.ParticleGun_single_ele_PtFlat20_100_etaFlatp23_43.recon.AOD.e5604_s3072_s3059_r8956            
9  mc15_14TeV.415013.ParticleGun_single_ele_PtFlat20_100_etaFlatp23_43.recon.AOD.e5604_s3072_s3059_r8956            

   datasetHash datasetScope  datasetScopeHash  \
0 -1271208501   mc15_5TeV   -448826083          
1  1880182958   mc15_14TeV  -2057006107         
2 -1777414577   mc15_14TeV  -2057006107         
3  1880182958   mc15_14TeV  -2057006107         
4 -1777414577   mc15_14TeV  -2057006107         
5 -1777414577   mc15_14TeV  -2057006107         
6 -1892801648   mc15_14TeV  -2057006107         
7 -1271208501   mc15_5TeV   -448826083          
8 -1331548957   mc15_14TeV  -2057006107         
9 -1331548957   mc15_14TeV  -2057006107         

                                dataset_version  dataset_versionHash datatype  \
0  e4962_s2891_s2921_r7899_r7676_tid08671884_00  948357197            AOD       
1  e4906_s2630_s2206_r7709                       1352491158           AOD       
2  e4906_s2630_s2206_r7768                      -1200349912           AOD       
3  e4906_s2630_s2206_r7709                       1352491158           AOD       
4  e4906_s2630_s2206_r7768                      -1200349912           AOD       
5  e4906_s2630_s2206_r7768                      -1200349912           AOD       
6  e2403_s3142_s3143_r9589                      -1199418070           AOD       
7  e4962_s2891_s2921_r7899_r7676_tid08671884_00  948357197            AOD       
8  e5604_s3072_s3059_r8956                      -976426417            AOD       
9  e5604_s3072_s3059_r8956                      -976426417            AOD       

   datatypeHash eventType  eventTypeHash eventVersion  eventVersionHash  \
0  1206945951    download  59378080       1.14.11     -2044360925         
1  1206945951    download  59378080       1.14.11     -2044360925         
2  1206945951    download  59378080       1.14.11     -2044360925         
3  1206945951    download  59378080       1.14.11     -2044360925         
4  1206945951    download  59378080       1.14.11     -2044360925         
5  1206945951    download  59378080       1.14.11     -2044360925         
6  1206945951    download  59378080       1.14.11     -2044360925         
7  1206945951    download  59378080       1.14.11     -2

In [109]:
datasets.describe().toPandas()

summary  account           accountHash         clientState  \
0  count   777286   777286                777286               
1  mean    None     1.935684591732567E8   None                 
2  stddev  None     1.1361317241087072E9  None                 
3  min     aagaard  -2127257360           CannotAuthenticate   
4  max     zzheng   2143592175            SourceNotFound       

        clientStateHash  \
0  777286                 
1  -9.74016245191591E8    
2  2.6212986830087802E8   
3  -1922357674            
4  1269532611             

                                                                                               dataset  \
0  777286                                                                                                
1  None                                                                                                  
2  None                                                                                                  
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456                                               
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210   

            datasetHash datasetScope      datasetScopeHash dataset_version  \
0  777286                777286       777286                777286           
1  1.4554549800768314E7  None         -7.552649492442358E8  None             
2  1.1941649925847783E9  None         8.314865501449602E8   None             
3  -2147401420           data10_hi    -2057006107           c1130_m1847      
4  2147445885            mc16_valid   2109319881            x557_m1947       

    dataset_versionHash datatype          datatypeHash eventType  \
0  777286                777286   777286                777286     
1  2.973310495739393E8   None     4.947060556405943E8   None       
2  1.2288188131163645E9  None     1.1700303292632787E9  None       
3  -2146716518           AOD      -2070622869           download   
4  2146405934            log      2124512252            download   

  eventTypeHash eventVersion      eventVersionHash  \
0  777286        777286       777286                 
1  5.937808E7    None         -1.992468944589716E9   
2  0.0           None         3.073273964746104E8    
3  59378080      1.13.2       -2044360925            
4  59378080      1.15.5       1261702107             

                             filename         filenameHash  \
0  777286                              777286                
1  None                                -248557.46481861244   
2  None                                1.240265814971077E9   
3  AOD.05371635._000001.pool.root.1    -2147478094           
4  log.13787275._000072.job.log.tgz.1  2147482617            

        filesize_double                hits                  hostname  \
0  777286                777286              777286                     
1  6.11402028756827E8    2.8777927300890536  None                       
2  1.2408771809945781E9  3.991629368416237   None                       
3  304.0                 1                   ReCaS-ui-01.cs.infn.it     
4  1.1991446515E10       48                  xenia.nevis.columbia.edu   

            hostnameHash prod_step          prod_stepHash     project  \
0  777286                 777286    777286                 777286       
1  -1.1804844919942595E8  None      -3.3290311308177426E7  None         
2  1.2286452227051136E9   None      1.1050073037449532E9   None         
3  -2133418459            PhysCont  -1329401808            data10_hi    
4  2129595856             simul     2037678380             mc16_valid   

            projectHash protocol          protocolHash             remoteSite  \
0  777286                777286   777286                777286                  
1  -7.551860864380562E8  None     3.5590511037907284E8  None                    
2  8.315047181989845E8   None     1.053620860192881E9   None                    
3  -2057006107           davs     -1607

In [110]:
joined6_temp = df_joined_check_6.select("SCOPE", "NAME", "CHILD_SCOPE", "CHILD_NAME","DID_TYPE","CHILD_TYPE","BYTES","UPDATED_AT","CREATED_AT","join_key_lookup")
# hier ist der fehler!! joined check 6 ist noch nicht gefiltert nach einzelnen datasets?
datasets.createOrReplaceTempView("datasets")
joined6_temp.createOrReplaceTempView("joined6_temp")
final_result = spark.sql("SELECT * \
                          FROM datasets LEFT JOIN joined6_final ON datasets.join_key_april_single == joined6_final.join_key_lookup")
final_result.describe().toPandas()

summary  account           accountHash         clientState  \
0  count   2246174  2246174               2246174              
1  mean    None     1.693082928236428E8   None                 
2  stddev  None     1.1393431230525322E9  None                 
3  min     aagaard  -2127257360           CannotAuthenticate   
4  max     zzheng   2143592175            SourceNotFound       

         clientStateHash  \
0  2246174                 
1  -1.0836076836655002E9   
2  3.900204761194457E8     
3  -1922357674             
4  1269532611              

                                                                                               dataset  \
0  2246174                                                                                               
1  None                                                                                                  
2  None                                                                                                  
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456                                               
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210   

             datasetHash datasetScope      datasetScopeHash dataset_version  \
0  2246174                2246174      2246174               2246174          
1  -1.5132742662009844E8  None         -8.025818049850082E8  None             
2  1.2023437923535213E9   None         8.406124066139916E8   None             
3  -2147401420            data10_hi    -2057006107           c1130_m1847      
4  2147445885             mc16_valid   2109319881            x557_m1947       

    dataset_versionHash datatype          datatypeHash eventType  \
0  2246174               2246174  2246174               2246174    
1  1.6535090439928606E8  None     3.2512745614884734E8  None       
2  1.2010950293127239E9  None     1.2247637496422014E9  None       
3  -2146716518           AOD      -2070622869           download   
4  2146405934            log      2124512252            download   

  eventTypeHash eventVersion       eventVersionHash  \
0  2246174       2246174      2246174                 
1  5.937808E7    None         -1.9969011611256185E9   
2  0.0           None         2.941301624686937E8     
3  59378080      1.13.2       -2044360925             
4  59378080      1.15.5       1261702107              

                             filename          filenameHash  \
0  2246174                             2246174                
1  None                                34314.13234771661      
2  None                                1.2460141111227674E9   
3  AOD.05371635._000001.pool.root.1    -2147478094            
4  log.13787275._000072.job.log.tgz.1  2147482617             

        filesize_double               hits                  hostname  \
0  2246174               2246174            2246174                    
1  1.1501053487418284E9  8.389489861426586  None                       
2  1.8764556109874809E9  9.693220985752982  None                       
3  304.0                 1                  ReCaS-ui-01.cs.infn.it     
4  1.1991446515E10       48                 xenia.nevis.columbia.edu   

            hostnameHash prod_step          prod_stepHash     project  \
0  2246174                2246174   2246174                2246174      
1  -1.7645940209908804E8  None      -1.6124881388248238E8  None         
2  1.1373798463162875E9   None      1.0688668002946571E9   None         
3  -2133418459            PhysCont  -1329401808            data10_hi    
4  2129595856             simul     2037678380             mc16_valid   

           projectHash protocol         protocolHash             remoteSite  \
0  2246174              2246174  2246174              2246174                 
1  -8.02442945080496E8  None     2.997944637918509E8  None                    
2  8.406518599016473E8  None     1.164795427796093E9  None                    
3  -2057006107          davs 

In [111]:
final_result.limit(10).toPandas()

account  accountHash clientState  clientStateHash  \
0  lhavener  1677896849   DONE       -878527175         
1  jsabater  1852428352   DONE       -878527175         
2  jsabater  1852428352   DONE       -878527175         
3  jsabater  1852428352   DONE       -878527175         
4  jsabater  1852428352   DONE       -878527175         
5  jsabater  1852428352   DONE       -878527175         
6  tofitsch  71670540     DONE       -878527175         
7  lhavener  1677896849   DONE       -878527175         
8  jsabater  1852428352   DONE       -878527175         
9  adurglis  813774196    DONE       -878527175         

                                                                                                          dataset  \
0  mc15_5TeV.420000.Hijing_PbPb_5p02TeV_MinBias_Flow_JJFV6.merge.AOD.e4962_s2891_s2921_r7899_r7676_tid08671884_00   
1  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768              
2  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709              
3  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768              
4  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768              
5  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768              
6  mc15_14TeV.147911.Pythia8_AU2CT10_jetjet_JZ1W.recon.AOD.e2403_s3142_s3143_r9589                                  
7  mc15_5TeV.420000.Hijing_PbPb_5p02TeV_MinBias_Flow_JJFV6.merge.AOD.e4962_s2891_s2921_r7899_r7676_tid08671884_00   
8  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709              
9  mc15_14TeV.415013.ParticleGun_single_ele_PtFlat20_100_etaFlatp23_43.recon.AOD.e5604_s3072_s3059_r8956            

   datasetHash datasetScope  datasetScopeHash  \
0 -1271208501   mc15_5TeV   -448826083          
1 -1777414577   mc15_14TeV  -2057006107         
2  1880182958   mc15_14TeV  -2057006107         
3 -1777414577   mc15_14TeV  -2057006107         
4 -1777414577   mc15_14TeV  -2057006107         
5 -1777414577   mc15_14TeV  -2057006107         
6 -1892801648   mc15_14TeV  -2057006107         
7 -1271208501   mc15_5TeV   -448826083          
8  1880182958   mc15_14TeV  -2057006107         
9 -1331548957   mc15_14TeV  -2057006107         

                                dataset_version  dataset_versionHash datatype  \
0  e4962_s2891_s2921_r7899_r7676_tid08671884_00  948357197            AOD       
1  e4906_s2630_s2206_r7768                      -1200349912           AOD       
2  e4906_s2630_s2206_r7709                       1352491158           AOD       
3  e4906_s2630_s2206_r7768                      -1200349912           AOD       
4  e4906_s2630_s2206_r7768                      -1200349912           AOD       
5  e4906_s2630_s2206_r7768                      -1200349912           AOD       
6  e2403_s3142_s3143_r9589                      -1199418070           AOD       
7  e4962_s2891_s2921_r7899_r7676_tid08671884_00  948357197            AOD       
8  e4906_s2630_s2206_r7709                       1352491158           AOD       
9  e5604_s3072_s3059_r8956                      -976426417            AOD       

   datatypeHash eventType  eventTypeHash eventVersion  eventVersionHash  \
0  1206945951    download  59378080       1.14.11     -2044360925         
1  1206945951    download  59378080       1.14.11     -2044360925         
2  1206945951    download  59378080       1.14.11     -2044360925         
3  1206945951    download  59378080       1.14.11     -2044360925         
4  1206945951    download  59378080       1.14.11     -2044360925         
5  1206945951    download  59378080       1.14.11     -2044360925         
6  1206945951    download  59378080       1.14.11     -2044360925         
7  1206945951    download  59378080       1.14.11     -2

In [93]:
df_merged_single = df_april.join(single_datasets, df_april.join_key_april == single_datasets.join_key_april_single, "left_outer")
df_merged_single.limit(10).toPandas()

account  accountHash clientState  clientStateHash  \
0  jsabater  1852428352   DONE       -878527175         
1  jsabater  1852428352   DONE       -878527175         
2  jsabater  1852428352   DONE       -878527175         
3  jsabater  1852428352   DONE       -878527175         
4  adurglis  813774196    DONE       -878527175         
5  adurglis  813774196    DONE       -878527175         
6  mmuskinj  1988900273   DONE       -878527175         
7  jsabater  1852428352   DONE       -878527175         
8  madesber -371588299    DONE       -878527175         
9  jsabater  1852428352   DONE       -878527175         

                                                                                                             dataset  \
0  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768                 
1  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768                 
2  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709                 
3  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768                 
4  mc15_14TeV.415013.ParticleGun_single_ele_PtFlat20_100_etaFlatp23_43.recon.AOD.e5604_s3072_s3059_r8956               
5  mc15_14TeV.415013.ParticleGun_single_ele_PtFlat20_100_etaFlatp23_43.recon.AOD.e5604_s3072_s3059_r8956               
6  mc16_13TeV.423000.ParticleGun_single_electron_egammaET.merge.AOD.e3566_s3113_r9397_r9315                            
7  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709                 
8  mc15_13TeV.402098.PythiaRhad_AUET2BCTEQ6L1_gen_gluino_p1_1400_qq_100_30ns.merge.AOD.e4620_s2770_s2183_r7772_r7676   
9  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768                 

   datasetHash datasetScope  datasetScopeHash                dataset_version  \
0 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768         
1 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768         
2  1880182958   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7709         
3 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768         
4 -1331548957   mc15_14TeV  -2057006107        e5604_s3072_s3059_r8956         
5 -1331548957   mc15_14TeV  -2057006107        e5604_s3072_s3059_r8956         
6  1673046285   mc16_13TeV  -1447489496        e3566_s3113_r9397_r9315         
7  1880182958   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7709         
8  607265341    mc15_13TeV   337785635         e4620_s2770_s2183_r7772_r7676   
9 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768         

   dataset_versionHash datatype  datatypeHash eventType  eventTypeHash  \
0 -1200349912           AOD      1206945951    download  59378080        
1 -1200349912           AOD      1206945951    download  59378080        
2  1352491158           AOD      1206945951    download  59378080        
3 -1200349912           AOD      1206945951    download  59378080        
4 -976426417            AOD      1206945951    download  59378080        
5 -976426417            AOD      1206945951    download  59378080        
6  58285954             AOD      1206945951    download  59378080        
7  1352491158           AOD      1206945951    download  59378080        
8  255664890            AOD      1206945951    download  59378080        
9 -1200349912           AOD      1206945951    download  59378080        

  eventVersion  eventVersionHash                          filename  \
0  1.14.11     -2044360925        AOD.08047211._000889.pool.root.1   
1  1.14.11     -2044360925        AOD.08047211._000874.pool.root.1   
2  1.14.11     -2044360925        AOD.07978428._000598.pool.root.1   
3  1.14.11     -2044360925        AOD.08047211._000283.pool.root.1   
4  1.14.1

In [94]:
single_datasets.count()

526956

In [95]:
df_merged_single.count()

777286

In [96]:
df_merged_single.agg(countDistinct("join_key_april_single")).toPandas()

count(DISTINCT join_key_april_single)
0  526956

In [97]:
df_lookup_renamed.createOrReplaceTempView("lookup_renamed")
df_lookup_renamed_stripped = spark.sql("SELECT SCOPE, NAME, CHILD_SCOPE, CHILD_NAME, DID_TYPE, CHILD_TYPE, LENGTH, BYTES, ADLER32, MD5, RULE_EVALUATION, UPDATED_AT, CREATED_AT, EVENTS, join_key_lookup\
                                        FROM lookup_renamed\
                                        WHERE SCOPE != 'panda' AND DID_TYPE == 'D'")
df_cl = df_merged_single.join(df_lookup_renamed_stripped, df_merged_single.join_key_april_single == df_lookup_renamed_stripped.join_key_lookup, "left_outer")
df_cl.limit(10).toPandas()

account  accountHash         clientState  clientStateHash  \
0  jsabater  1852428352   DONE               -878527175         
1  jsabater  1852428352   DONE               -878527175         
2  jsabater  1852428352   DONE               -878527175         
3  jsabater  1852428352   DONE               -878527175         
4  adurglis  813774196    DONE               -878527175         
5  adurglis  813774196    DONE               -878527175         
6  jsabater  1852428352   DONE               -878527175         
7  madesber -371588299    DONE               -878527175         
8  jsabater  1852428352   DONE               -878527175         
9  mmadugod  364580267    ServiceUnavailable -1394468120        

                                                                                                             dataset  \
0  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768                 
1  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768                 
2  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709                 
3  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768                 
4  mc15_14TeV.415013.ParticleGun_single_ele_PtFlat20_100_etaFlatp23_43.recon.AOD.e5604_s3072_s3059_r8956               
5  mc15_14TeV.415013.ParticleGun_single_ele_PtFlat20_100_etaFlatp23_43.recon.AOD.e5604_s3072_s3059_r8956               
6  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709                 
7  mc15_13TeV.402098.PythiaRhad_AUET2BCTEQ6L1_gen_gluino_p1_1400_qq_100_30ns.merge.AOD.e4620_s2770_s2183_r7772_r7676   
8  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768                 
9  mc15_14TeV.427080.Pythia8EvtGen_A14NNPDF23LO_flatpT_Zprime.recon.AOD.e6172_s3142_s3143_r9589                        

   datasetHash datasetScope  datasetScopeHash                dataset_version  \
0 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768         
1 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768         
2  1880182958   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7709         
3 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768         
4 -1331548957   mc15_14TeV  -2057006107        e5604_s3072_s3059_r8956         
5 -1331548957   mc15_14TeV  -2057006107        e5604_s3072_s3059_r8956         
6  1880182958   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7709         
7  607265341    mc15_13TeV   337785635         e4620_s2770_s2183_r7772_r7676   
8 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768         
9  883787055    mc15_14TeV  -2057006107        e6172_s3142_s3143_r9589         

   dataset_versionHash datatype  datatypeHash eventType  eventTypeHash  \
0 -1200349912           AOD      1206945951    download  59378080        
1 -1200349912           AOD      1206945951    download  59378080        
2  1352491158           AOD      1206945951    download  59378080        
3 -1200349912           AOD      1206945951    download  59378080        
4 -976426417            AOD      1206945951    download  59378080        
5 -976426417            AOD      1206945951    download  59378080        
6  1352491158           AOD      1206945951    download  59378080        
7  255664890            AOD      1206945951    download  59378080        
8 -1200349912           AOD      1206945951    download  59378080        
9 -1677568547           AOD      1206945951    download  59378080        

  eventVersion  eventVersionHash                          filename  \
0  1.14.11     -2044360925        AOD.08047211._000889.pool.root.1   
1  1.14.11     -2044360925        AOD.08047211._000874.pool.root.1   
2  1.14.11     -2044360925        AOD.07978428._000598.pool.r

In [98]:
df_lookup_renamed_stripped.agg(*(countDistinct(col(c)).alias(c) for c in df_lookup_renamed_stripped.columns)).toPandas()

SCOPE     NAME  CHILD_SCOPE  CHILD_NAME  DID_TYPE  CHILD_TYPE  LENGTH  \
0  2438   7590137  2263         748310955   1         1           1        

       BYTES    ADLER32   MD5  RULE_EVALUATION  UPDATED_AT  CREATED_AT  \
0  248292497  684865085  1230  2                99451511    104622524    

   EVENTS  join_key_lookup  
0  486057  748337890

In [99]:
df_joined_check_6.agg(*(countDistinct(col(c)).alias(c) for c in df_joined_check_6.columns)).toPandas()

account  accountHash  clientState  clientStateHash  dataset  datasetHash  \
0  539      539          11           11               10574    10574         

   datasetScope  datasetScopeHash  dataset_version  dataset_versionHash  \
0  23            23                3173             3173                  

   datatype  datatypeHash  eventType  eventTypeHash  eventVersion  \
0  112       112           1          1              5              

   eventVersionHash  filename  filenameHash  filesize_double  hits  hostname  \
0  5                 530096    530054        469129           38    344        

   hostnameHash  prod_step  prod_stepHash  project  projectHash  protocol  \
0  344           8          8              22       22           6          

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0  6             164         164             3434        3434             

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0  22         2954         2954             776980   776904       327       

   traceIpHash  traceTimeentryUnix  transferStart   uuid  uuidHash  \
0  327          776978              776978         12873  12873      

   join_key_april  fileScope  SCOPE   NAME  CHILD_SCOPE  CHILD_NAME  DID_TYPE  \
0  530096          22         36     12704  22           530096      1          

   CHILD_TYPE   BYTES  UPDATED_AT  CREATED_AT  join_key_lookup  
0  1           469129  229218      433065      530096

In [100]:
df_merged_single.agg(*(countDistinct(col(c)).alias(c) for c in df_merged_single.columns)).toPandas()

account  accountHash  clientState  clientStateHash  dataset  datasetHash  \
0  539      539          11           11               10578    10578         

   datasetScope  datasetScopeHash  dataset_version  dataset_versionHash  \
0  23            23                3175             3175                  

   datatype  datatypeHash  eventType  eventTypeHash  eventVersion  \
0  112       112           1          1              5              

   eventVersionHash  filename  filenameHash  filesize_double  hits  hostname  \
0  5                 530100    530058        469133           38    344        

   hostnameHash  prod_step  prod_stepHash  project  projectHash  protocol  \
0  344           8          8              22       22           6          

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0  6             164         164             3435        3435             

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0  22         2954         2954             776984   776908       327       

   traceIpHash  traceTimeentryUnix  transferStart   uuid  uuidHash  \
0  327          776982              776982         12877  12877      

   join_key_april  fileScope  join_key_april_single  distinct_datasets  
0  530100          22         526956                 1

In [101]:
df_april.count()

777286

In [102]:
df_cl.count()

777319

In [112]:
df_cl.write.format("json").save("/user/sprenner/df_cl.json")

In [103]:
df_cl.groupBy().sum('distinct_datasets').toPandas()

sum(distinct_datasets)
0  773511

In [104]:
df_merged_single.createOrReplaceTempView("merged_single")
tt = spark.sql("SELECT COUNT(DISTINCT join_key_april_single)\
                FROM merged_single")
tt.toPandas()

count(DISTINCT join_key_april_single)
0  526956

In [105]:
df_joined_check_6.describe().toPandas()

summary  account           accountHash         clientState  \
0  count   782618   782618                782618               
1  mean    None     1.9731572582537585E8  None                 
2  stddev  None     1.1370231297971792E9  None                 
3  min     aagaard  -2127257360           CannotAuthenticate   
4  max     zzheng   2143592175            SourceNotFound       

        clientStateHash  \
0  782618                 
1  -9.74144880447121E8    
2  2.6189898366317788E8   
3  -1922357674            
4  1269532611             

                                                                                               dataset  \
0  782618                                                                                                
1  None                                                                                                  
2  None                                                                                                  
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r2111_p456                                               
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_gammajet_DP280_500.merge.AOD.e5094_s3238_r10441_r10210   

            datasetHash datasetScope      datasetScopeHash dataset_version  \
0  782618                782618       782618                782618           
1  1.887588126440869E7   None         -7.555467303006039E8  None             
2  1.1947515338903456E9  None         8.327686484635468E8   None             
3  -2147401420           data10_hi    -2057006107           c1130_m1847      
4  2147445885            mc16_valid   2109319881            x557_m1947       

    dataset_versionHash datatype         datatypeHash eventType eventTypeHash  \
0  782618                782618   782618               782618    782618         
1  2.9641383972466004E8  None     4.950545094142839E8  None      5.937808E7     
2  1.2265050444455762E9  None     1.167857100188329E9  None      0.0            
3  -2146716518           AOD      -2070622869          download  59378080       
4  2146405934            log      2124512252           download  59378080       

  eventVersion      eventVersionHash                            filename  \
0  782618       782618                782618                               
1  None         -1.989902781594934E9  None                                 
2  None         3.146099197703222E8   None                                 
3  1.13.2       -2044360925           AOD.05371635._000001.pool.root.1     
4  1.15.5       1261702107            log.13787275._000072.job.log.tgz.1   

           filenameHash       filesize_double                hits  \
0  782618                782618                782618               
1  -567794.2139920114    6.143201947233235E8   2.870071989144129    
2  1.2403538392366462E9  1.2407176955045416E9  3.9800985717626576   
3  -2147478094           304.0                 1                    
4  2147482617            1.1991446515E10       48                   

                   hostname          hostnameHash prod_step  \
0  782618                    782618                782618     
1  None                      -1.201348405637169E8  None       
2  None                      1.2267734034224434E9  None       
3  ReCaS-ui-01.cs.infn.it    -2133418459           PhysCont   
4  xenia.nevis.columbia.edu  2129595856            simul      

           prod_stepHash     project           projectHash protocol  \
0  782618                 782618      782618                782618    
1  -2.7711860624417022E7  None        -7.553693460539573E8  None      
2  1.1064683221740837E9   None        8.328094994956783E8   None      
3  -1329401808            data10_hi   -2057006107           davs      
4  2037678380             mc16_valid  2109319881            srm       

           protocolHash             remoteSite        remoteSiteHash  \
0  782618                782618                 782618                 
1  3.5774461368790907E8  None          

In [106]:
april = df_joined_check_6.selectExpr('account','accountHash','clientState','clientStateHash','dataset as dataset_old','datasetHash as datasetHash_old',
                                     'datasetScope as datasetScope_old','datasetScopeHash as datasetScopeHash_old','dataset_version','dataset_versionHash','datatype',
                                     'datatypeHash','eventType','eventTypeHash','eventVersion','eventVersionHash','filename',
                                     'filenameHash','filesize_double','hits','hostname','hostnameHash','prod_step','prod_stepHash',
                                     'project','projectHash','protocol','protocolHash','remoteSite','remoteSiteHash','run_number',
                                     'run_numberHash','fileScope','scopeHash as fileScopeHash','stream_name','stream_nameHash','traceId','traceIdHash','traceIp',
                                     'traceIpHash','traceTimeentryUnix','transferStart','uuid','uuidHash', "join_key_april")
look = df_joined_check_6.selectExpr("SCOPE", "NAME", "CHILD_SCOPE", "CHILD_NAME", "DID_TYPE", "CHILD_TYPE", "BYTES", "UPDATED_AT", "CREATED_AT", "join_key_lookup")